This file outlines the method for creating an **embedding based** question answering system.
Following are the major steps followed
* Find the entity
* Create a set of core chain candidates
* Rank the core chain candidate

In [2]:
# imports (external library)
import json
import numpy
import requests
from pprint import pprint
import create_data_node as cdn
from utils import natural_language_utilities as nlutils

In [3]:
# predicate blacklist to exclude some of the meta data information
pb = open('resources/predicate.blacklist').readlines()
pb[-1] = pb[-1] + '\n'
pb = [r[:-1] for r in pb]

#### Entity linking

We employ [EARL](http://sda.cs.uni-bonn.de/projects/earl/) which returns a set of candidate entities of which we use the top most one. 

In [4]:
def get_entities(question,show_internals=False):
    """
        uses EARL to find all the entites present in the question.
        :param question: a natural language question.
        :return: entities list.
    """

    headers = {
        'Content-Type': 'application/json',
    }

    data = '{"nlquery":"%(p)s"}'% {"p":question}
    response = requests.post(' http://asknow02.sda.tech/earl/api/processQuery', headers=headers, data=data)
    a = json.loads(response.content)
    if show_internals:
        pprint(a)
    entity_list = []
    for i in range(len(a['ertypes'])):
        if a['ertypes'][i] == 'entity':
            entity_list.append(a['rerankedlists'][str(i)][0][1]) # return the top most one. 
    return entity_list

print(get_entities('Who is the president of India ?',False))

['http://dbpedia.org/resource/India']


In [5]:

cd_node = cdn.CreateDataNode(_predicate_blacklist=pb, _relation_file={}, _qald=False)

Label Cache not found. Creating a new one


Traceback (most recent call last):
  File "/home/gaurav/codes/tutorial/QA-Tutorial/utils/dbpedia_interface.py", line 137, in __init__
    self.labels = pickle.load(open('resources/labels.pickle'))
TypeError: a bytes-like object is required, not 'str'


In [6]:
hop1,hop2  = cd_node.create_subgraph.subgraph\
            (['http://dbpedia.org/resource/Michael_Crichton'],[],_use_blacklist=True,_qald=False)

if False:
    print('few examples of hop1 candidates')
    pprint(hop1[:5])
    print('few examples of hop2 candidates')
    pprint(hop2[:5])

URLError: <urlopen error [Errno 111] Connection refused>

In [ ]:
# nlutils.get_label_via_parsing(hop1[1][1])
#             query = {'question':p}
#             v = requests.get("http://localhost:3500/vec", json=query)
#             v = np.asarray(v.json())
#             v_p = np.mean(v.astype(np.float), axis=0)

#             # If either of them is a zero vector, the cosine is 0.\
#             if np.sum(v_p) == 0.0 or np.sum(_v_qt) == 0.0 or p.strip() == "":
#                 similarity_arr[i] = np.float64(0.0)
#                 continue
#             try:
#                 # Cos Product
#                 similarity_arr[i] = np.dot(v_p, _v_qt) / (np.linalg.norm(v_p) * np.linalg.norm(_v_qt))
#             except:
#                 traceback.print_exc()

In [7]:
def assign_score(core_chain,question):
    if len(core_chain) == 2:
        # corechain looks like ['+', 'http:.../abc']
        predicate = nlutils.get_label_via_parsing(core_chain[1],lower=True)
    else:
        # corechain looks like ['+' 'http:../abc', '-', 'http:../pqr']
        predicate = [nlutils.get_label_via_parsing(core_chain[1],lower=True),
                     nlutils.get_label_via_parsing(core_chain[3],,lower=True)]
        predicate = " ".join(predicate)
    
    # predicate is a string seperated by space -- 'abc pqr'
    

SyntaxError: invalid syntax (<ipython-input-7-6e5a6db55eb2>, line 8)

In [8]:
query = {'question':'Who is the president of India ?'}
v = requests.get("http://localhost:3500/vec", json=query)
v = np.asarray(v.json())

ConnectionError: HTTPConnectionPool(host='localhost', port=3500): Max retries exceeded with url: /vec (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa91c46b160>: Failed to establish a new connection: [Errno 111] Connection refused',))

#### Loading dataset. 

The dataset consists of 2000 questions without any rdf constraint or count or ask. It is just composed of single hop or two hop query.

In [9]:
dataset = json.load(open('resources/dataset_with_paths.json'))